NAME: YASH PATNI

ROLL NO.: A4-71

PRACTICAL NO. 3

AIM:

(A) Write a program to find FIRST for any grammar. All the following rules of FIRST must be implemented.

(B) Calculate Follow for the given grammar and Construct the LL (1) parsing table using the FIRST and FOLLOW.

In [ ]:
import sys
from tabulate import tabulate

terminals = ['a', 'b', 'c', 'p']
non_terminals = ['S', 'A', 'B', 'C']
starting_symbol = 'S'
productions = ["S->ABC/C", "A->a/bB/#", "B->p/#", "C->c"]
productions_dict = {nT: [] for nT in non_terminals}

for production in productions:
    nonterm_to_prod = production.split("->")
    alternatives = nonterm_to_prod[1].split("/")
    for alternative in alternatives:
        productions_dict[nonterm_to_prod[0]].append(alternative)

LL1_table = {}
conflicts = False

def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]
        for alternative in alternatives:
            first_2 = first(alternative)
            first_ |= first_2
    elif string in terminals:
        first_ = {string}
    elif string == '' or string == '#':
        first_ = {'#'}
    else:
        first_2 = first(string[0])
        if '#' in first_2:
            i = 1
            while '#' in first_2:
                first_ |= (first_2 - {'#'})
                if string[i:] in terminals:
                    first_ |= {string[i:]}
                    break
                elif string[i:] == '':
                    first_ |= {'#'}
                    break
                first_2 = first(string[i:])
                first_ |= (first_2 - {'#'})
                i += 1
        else:
            first_ |= first_2
    return first_

def follow(nT):
    follow_ = set()
    prods = productions_dict.items()
    if nT == starting_symbol:
        follow_ |= {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for char in alt:
                if char == nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ |= follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if '#' in follow_2:
                            follow_ |= (follow_2 - {'#'})
                            follow_ |= follow(nt)
                        else:
                            follow_ |= follow_2
    return follow_

FIRST = {non_terminal: set() for non_terminal in non_terminals}
FOLLOW = {non_terminal: set() for non_terminal in non_terminals}

for non_terminal in non_terminals:
    FIRST[non_terminal] |= first(non_terminal)

FOLLOW[starting_symbol] |= {'$'}
for non_terminal in non_terminals:
    FOLLOW[non_terminal] |= follow(non_terminal)

print("{: ^20}{: ^20}{: ^20}".format('Non Terminals', 'First', 'Follow'))
for non_terminal in non_terminals:
    print("{: ^20}{: ^20}{: ^20}".format(non_terminal, str(FIRST[non_terminal]), str(FOLLOW[non_terminal])))

for non_terminal, alternatives in productions_dict.items():
    for alternative in alternatives:
        first_set_alt = first(alternative)
        for terminal in first_set_alt - {'#'}:
            if (terminal, non_terminal) not in LL1_table:
                LL1_table[(terminal, non_terminal)] = [alternative]
            else:
                LL1_table[(terminal, non_terminal)].append(alternative)
                if len(LL1_table[(terminal, non_terminal)]) > 1:
                    conflicts = True
                    print(f"Conflict at ({terminal}, {non_terminal})")

        if '#' in first_set_alt or '' in first_set_alt:  # If epsilon is in FIRST(α)
            for terminal in FOLLOW[non_terminal]:
                if (terminal, non_terminal) not in LL1_table:
                    LL1_table[(terminal, non_terminal)] = [alternative]
                else:
                    LL1_table[(terminal, non_terminal)].append(alternative)
                    if len(LL1_table[(terminal, non_terminal)]) > 1:
                        conflicts = True
                        print(f"Conflict at ({terminal}, {non_terminal})")

# Displaying the LL(1) parsing table with terminals as row headers and non-terminals as column headers
table_data = []
for terminal in terminals:
    row = [terminal] + [', '.join(LL1_table.get((terminal, non_terminal), [])) for non_terminal in non_terminals]
    table_data.append(row)

print(tabulate(table_data, headers=['Terminals'] + non_terminals, tablefmt='grid'))

if conflicts:
    print("Conflicts found.")
else:
    print("No conflicts found.")


   Non Terminals           First               Follow       
         S          {'a', 'p', 'b', 'c'}       {'$'}        
         A            {'a', 'b', '#'}        {'p', 'c'}     
         B               {'p', '#'}          {'p', 'c'}     
         C                 {'c'}               {'$'}        
Conflict at (c, S)
Conflict at (p, B)
+-------------+--------+-----+------+-----+
| Terminals   | S      | A   | B    | C   |
+=============+========+=====+======+=====+
| a           | ABC    | a   |      |     |
+-------------+--------+-----+------+-----+
| b           | ABC    | bB  |      |     |
+-------------+--------+-----+------+-----+
| c           | ABC, C | #   | #    | c   |
+-------------+--------+-----+------+-----+
| p           | ABC    | #   | p, # |     |
+-------------+--------+-----+------+-----+
Conflicts found.


In [ ]:
from collections import defaultdict
from tabulate import tabulate


class LLParser:
    def __init__(self, productions, first_sets, follow_sets):
        self.productions = productions
        self.first = first_sets
        self.follow = follow_sets
        self.start_symbol = list(self.productions.keys())[0]
        self.construct_parsing_table()

    def construct_parsing_table(self):
        terminals = set()
        non_terminals = set(self.productions.keys())
        for nt, rhs in self.productions.items():
            for alt in rhs:
                for symbol in alt:
                    if not symbol.isupper() and symbol != 'e':
                        terminals.add(symbol)
        terminals.add('$')

        self.parsing_table = defaultdict(lambda: defaultdict(str))
        for nt in non_terminals:
            for t in terminals:
                for prod in self.productions[nt]:
                    for i, letter in enumerate(prod):
                        if prod == 'e' and (t in self.follow[nt] or t == '$'):
                            self.parsing_table[nt][t] = f"{nt}->e"
                            break
                        elif letter.isupper() and 'e' in self.first[letter] and len(prod) > i + 1 and t in self.first[prod[i + 1]]:
                            self.parsing_table[nt][t] = f"{nt}->{prod}"
                            break
                        elif prod[0] == t or (prod[0].isupper() and t in self.first[prod[0]]):
                            self.parsing_table[nt][t] = f"{nt}->{prod}"
                            break
                        elif letter.isupper() and 'e' in self.first[letter] and len(prod) > i + 1 and t in self.first[prod[i + 1]]:
                            self.parsing_table[nt][t] = f"{nt}->{prod}"
                            if 'e' not in self.first[letter]:
                                break
                    else:
                        if prod == 'B' and t in self.follow[nt] and 'c' in self.first['C']:
                            self.parsing_table[nt][t] = f"{nt}->e"

    def print_parsing_table(self):
        headers = sorted(list(self.parsing_table['S'].keys()))
        rows = sorted(list(self.productions.keys()))
        table = [[self.parsing_table[nt][t] for t in headers] for nt in rows]
        print(tabulate(table, headers=headers, showindex=rows, tablefmt='grid'))


def main():
    a2 = {
        'S': ['ABC', 'C'],
        'A': ['a', 'bB', 'e'],
        'B': ['p', 'e'],
        'C': ['c']
    }

    first_sets = {
        'S': {'a', 'b', 'c', 'p'},
        'A': {'a', 'b', 'e'},
        'B': {'p', 'e'},
        'C': {'c'}
    }

    follow_sets = {
        'S': {'$'},
        'A': {'c', 'p'},
        'B': {'c', 'p'},
        'C': {'$'}
    }

    parser = LLParser(a2, first_sets, follow_sets)
    parser.print_parsing_table()


if __name__ == "__main__":
    main()
